In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
#https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_(Physical_and_Theoretical_Chemistry)/Quantum_Mechanics/05.5%3A_Particle_in_Boxes/Particle_in_a_2-Dimensional_Box
def true_2D_SD(xy): 
    y = 2 * np.sin(np.pi*xy[:,0].reshape(-1,1))*np.sin(np.pi*xy[:,1].reshape(-1,1))  
    
    return y

In [6]:
loss_thresh = 0.1
label = "SD_stan"

x = np.linspace(0,1,500).reshape(-1,1)
y = np.linspace(0,1,500).reshape(-1,1)

X,Y = np.meshgrid(x,y)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
  
xy = np.hstack((X,Y))

u_true = true_2D_SD(xy)
u_true_norm = np.linalg.norm(u_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]
pi = torch.from_numpy(np.array(np.pi).reshape(-1,)).float().to(device)

xy_test_tensor = torch.from_numpy(xy).float().to(device)


lb_xy = xy[0]
ub_xy = xy[-1]

In [7]:
def trainingdata(N_T,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    N_t = int(N_T/4)
    
    x_BC1 = np.random.uniform(size = N_t).reshape(-1,1)
    y_BC1 = np.zeros((N_t,1))
    u_BC1 = 0*np.ones((N_t,1))
    
    x_BC2 = np.ones((N_t,1))
    y_BC2 = np.random.uniform(size = N_t).reshape(-1,1) 
    u_BC2 = 0*np.ones((N_t,1))
    
    x_BC3 = np.random.uniform(size = N_t).reshape(-1,1)
    y_BC3 = np.ones((N_t,1)) 
    u_BC3 = 0*np.ones((N_t,1))
    
    x_BC4 = np.zeros((N_t,1))
    y_BC4 = np.random.uniform(size = N_t).reshape(-1,1) 
    u_BC4 = 0*np.ones((N_t,1))
    
    XY_corners = np.array([[0,0],[1,0],[0,1],[1,1]]).reshape(-1,2)
    U_corners = 0*np.ones((4,1))
    
    XY_1 = np.hstack((x_BC1,y_BC1))
    XY_2 = np.hstack((x_BC2,y_BC2))
    XY_3 = np.hstack((x_BC3,y_BC3))
    XY_4 = np.hstack((x_BC4,y_BC4))
    
    xy_BC = np.vstack((XY_1,XY_2,XY_3,XY_4,XY_corners)) #choose indices from  set 'idx' (x,t)
    u_BC = np.vstack((u_BC1,u_BC2,u_BC3,u_BC4,U_corners))
    
    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xy_coll = lb_xy + (ub_xy - lb_xy)*samples
    
    xy_coll = np.vstack((xy_coll, xy_BC)) # append training points to collocation points 

    return xy_coll, xy_BC, u_BC

In [31]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

            
    'foward pass'
    def forward(self,xy):
        if torch.is_tensor(xy) != True:         
            xy = torch.from_numpy(xy)                
        
        ubxy = torch.from_numpy(ub_xy).float().to(device)
        lbxy = torch.from_numpy(lb_xy).float().to(device)
    
                      
        #preprocessing input 
        xy = (xy - lbxy)/(ubxy - lbxy)
        
        #convert to float
        a = xy.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z)
            a = z1 + self.beta[:,i]*z*z1            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xy,u):
                
        loss_bc = self.loss_function(self.forward(xy), u)
                
        return loss_bc
    
    def loss_PDE(self, xy_coll, f_hat):
        
        g = xy_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        
        u_x_y = autograd.grad(u,g,torch.ones([xy_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy = autograd.grad(u_x_y,g,torch.ones(xy_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_yy[:,[0]]
        d2u_dy2 = u_xx_yy[:,[1]]    
        

        f = (1/2)*(d2u_dx2+d2u_dy2) + torch.pow(pi,2)*u         
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xy_BC,y_BC,xy_coll,f_hat):

        loss_BC = self.loss_BC(xy_BC,y_BC)
        loss_f = self.loss_PDE(xy_coll,f_hat)
        
        loss_val = 100000000*loss_BC + 100000000000*loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xy_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re  

In [23]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [24]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
#     xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,seed*123)
#     xt_coll = torch.from_numpy(xt_coll).float().to(device)
#     xt_BC = torch.from_numpy(xt_BC).float().to(device)
#     y_BC = torch.from_numpy(y_BC).float().to(device)

#     f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [25]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*9)
    start_time = time.time() 
    thresh_flag = 0

    xy_coll, xy_BC, u_BC = trainingdata(N_T,N_f,rep*11)
    xy_coll = torch.from_numpy(xy_coll).float().to(device)
    xy_BC = torch.from_numpy(xy_BC).float().to(device)
    u_BC = torch.from_numpy(u_BC).float().to(device)

    f_hat = torch.zeros(xy_coll.shape[0],1).to(device)
    

    for i in range(max_iter):
        train_step(xy_BC, u_BC, xy_coll,f_hat,i)
        loss_np = PINN.loss(xy_BC, u_BC, xy_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1          
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])   
        
      
         

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [32]:
max_reps = 1 #10
max_iter = 100 #200

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

beta_init = 1

N_T = 5000
N_f = 10000 #Total number of collocation points

for reps in range(max_reps):
    print(label)
    print(reps)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []
    beta_val = []

    torch.manual_seed(reps*36)

    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    #layers = np.array([2,50,50,50,50,50,50,50,1])

    PINN = Sequentialmodel(layers,beta_init)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




    nan_flag = train_model(max_iter,reps)



    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    beta_full.append(beta_val)


  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

SD_stan
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 465895500.0 Test MSE 1.0098465424490477 Test RE 1.006925061349366
1 Train Loss 31665684.0 Test MSE 0.9786976971919796 Test RE 0.9912740608680952
2 Train Loss 8612959.0 Test MSE 0.9672348654801763 Test RE 0.9854518977553207
3 Train Loss 3892501.8 Test MSE 0.9690149853153179 Tes

96 Train Loss 4756.1074 Test MSE 0.995018537782908 Test RE 0.9995051696098822
97 Train Loss 4756.1074 Test MSE 0.995018537782908 Test RE 0.9995051696098822
98 Train Loss 4756.1074 Test MSE 0.995018537782908 Test RE 0.9995051696098822
99 Train Loss 4756.1074 Test MSE 0.995018537782908 Test RE 0.9995051696098822
Training time: 121.33


In [ ]:
nan_tune

In [ ]:
import scipy.io as sio
import numpy as np

In [ ]:
for tune_reps in range(20):
    label = "KG_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

In [ ]:
a = 0
for i in range(10):
    a = a+ test_re_full[i][-1]
    print(test_re_full[i][-1])
    
print("a = ",a/10)